In [1]:
%load_ext autoreload
%autoreload 2

import jax
import jax.numpy as jnp
import flax.linen as nn
from flax import struct

from jaxav.dynamics import DroneState, step
from jaxav.pytree import tree_stack, to_numpy
from jaxav.visualization import render
from jaxav.control import RateController, PositionController
from jaxav.envs.hover import Hover
from jaxav.base import RolloutWrapper

from tqdm import tqdm
import time

import yaml
drone = "jaxav/asset/hummingbird.yaml"
with open(drone, "r") as f:
    params = yaml.safe_load(f)

key = jax.random.PRNGKey(0)

2023-06-28 23:01:27.598570: W external/xla/xla/service/gpu/nvptx_compiler.cc:596] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.1.105). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
env = Hover(drone)

class Foo(nn.Module):

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(128)(x)
        x = jnp.sum(x)
        return x



FrozenDict({
    params: {
        kernel: Array([[ 0.36402658,  0.17710859, -0.16770986, ...,  0.40100864,
                -0.47886357,  0.07992616],
               [ 0.42209762,  0.11826899,  0.11172286, ...,  0.01486722,
                 0.26404345, -0.26247606],
               [-0.39979404,  0.48579267,  0.12676321, ...,  0.17139047,
                -0.28431538, -0.10210536],
               ...,
               [-0.2940564 ,  0.40860546,  0.15569891, ...,  0.25664407,
                -0.11676306,  0.3151261 ],
               [-0.39524713, -0.4403522 , -0.15616548, ...,  0.09872915,
                 0.04761202, -0.13373263],
               [ 0.37237784, -0.08677061, -0.22721125, ...,  0.1556863 ,
                 0.0919781 ,  0.27768564]], dtype=float32),
        bias: Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0